In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [42.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchp

In [2]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [3]:
# create dataframe
sentenceData = spark.createDataFrame([
    (0, ['Big', 'data', 'is', 'super', 'powerful']),
    (1, ['This', 'is', 'going', 'to', 'be', 'epic'])
], ['id', 'raw'])
sentenceData.show(truncate= False)

+---+--------------------------------+
|id |raw                             |
+---+--------------------------------+
|0  |[Big, data, is, super, powerful]|
|1  |[This, is, going, to, be, epic] |
+---+--------------------------------+



In [23]:
#import stop words library
from pyspark.ml.feature import StopWordsRemover
# import tokenizer library
from pyspark.ml.feature import Tokenizer
# to create udf count column
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline

In [5]:
# run the remover
remover = StopWordsRemover(inputCol= "raw", outputCol= "filtered")

In [6]:
# transform and show data
remover.transform(sentenceData).show(truncate = False)

+---+--------------------------------+----------------------------+
|id |raw                             |filtered                    |
+---+--------------------------------+----------------------------+
|0  |[Big, data, is, super, powerful]|[Big, data, super, powerful]|
|1  |[This, is, going, to, be, epic] |[going, epic]               |
+---+--------------------------------+----------------------------+



In [25]:
# tokenize and remove stop words on df
# create the df
df = spark.createDataFrame([
    (0, "We are going to now tokenize and remove stop words from this dataframe"),
    (1, "First we have break these sentences up into tokens, or single words"),
    (3, "Then we will remove all of the stop words from the sentences")                       
], ["id", "sentence"])


In [12]:
# create the tokenizer
tokenizer = Tokenizer(inputCol= "sentence", outputCol= "words")

In [13]:
# Create the remover
remover = StopWordsRemover(inputCol= "words", outputCol= "filtered")

In [26]:
# create pipeline
pipeline = Pipeline(stages=[tokenizer, remover])

In [30]:
model = pipeline.fit(df)
df2 = model.transform(df)
df2.show(truncate= False)

+---+----------------------------------------------------------------------+------------------------------------------------------------------------------------+-------------------------------------------------+
|id |sentence                                                              |words                                                                               |filtered                                         |
+---+----------------------------------------------------------------------+------------------------------------------------------------------------------------+-------------------------------------------------+
|0  |We are going to now tokenize and remove stop words from this dataframe|[we, are, going, to, now, tokenize, and, remove, stop, words, from, this, dataframe]|[going, tokenize, remove, stop, words, dataframe]|
|1  |First we have break these sentences up into tokens, or single words   |[first, we, have, break, these, sentences, up, into, tokens,, or, single, wo

In [19]:
df = tokenizer.transform(df)
df = remover.transform(df)
df.show(truncate= False)

+---+----------------------------------------------------------------------+------------------------------------------------------------------------------------+-------------------------------------------------+
|id |sentence                                                              |words                                                                               |filtered                                         |
+---+----------------------------------------------------------------------+------------------------------------------------------------------------------------+-------------------------------------------------+
|0  |We are going to now tokenize and remove stop words from this dataframe|[we, are, going, to, now, tokenize, and, remove, stop, words, from, this, dataframe]|[going, tokenize, remove, stop, words, dataframe]|
|1  |First we have break these sentences up into tokens, or single words   |[first, we, have, break, these, sentences, up, into, tokens,, or, single, wo